Predicción de la evolución de la diabetes en pacientes usando MLP
===

Se desea construir um modelo de regresión no lineal (redes neuronales artificiales) que permita pronósticar el progreso de la diabetes con un horizonte de doce meses con base en variables físicas y pruebas de laboratorio. Véase https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html

En este problema se tiene una base de datos de diez variables base (edad, sexo, índice de masa corporal, presión arterial, y seis variables medidas en sangre) para 442 pacientes, y un índice que mide el progreso de la diabetes un año después de la prueba. La columna Y es la variable explicada.

In [20]:
#
# La muestra se encuentra dividida en tres partes:
#
#   * X_train, y_true_train: es la muestra para estimar los parametros optimos
#
#   * X_test, y_true_test: es la muestra para seleccionar la mejor configuración
#
#   * X_val, y_true_val: es la muestra para probar el modelo en productivo
#
#
# Rta/
# True
#

import warnings

import pandas as pd
import pytest
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")

df = pd.read_csv("https://raw.githubusercontent.com/jdvelasq/datalabs/master/datasets/diabetes.csv")

##Colocar todas las columnas del dataset en escala de 0 a 1

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 0.5))
df_scaler = scaler.fit_transform(df)
name_columns = list(df.columns)
new_df = pd.DataFrame(df_scaler, columns = name_columns)

## Establecer mis variables X y y a partir del dataset transformado
y= new_df['target']
X= new_df.drop(['target'], axis=1)

## Particionamiento de los datos en 70% entrenamiento, 15% prueba y 15% validación en productivo.
from sklearn.model_selection import train_test_split

X_train, X_test, y_true_train, y_true_test = train_test_split(
    X, 
    y, 
    test_size=0.3, 
    random_state=177
)

X_test, X_val, y_true_test, y_true_val = train_test_split(
    X_test, 
    y_true_test, 
    test_size=0.5, 
    random_state=177
)
## Encontrar parámetros óptimos en modelo MLPRegressor
from sklearn.model_selection import GridSearchCV

param_grid = [
    {
        "hidden_layer_sizes": [(1,),(2,),(3,),(4,),(5,)],
        "random_state": [1000,1001,1002,1003,1004,1005]
    }
]

m = MLPRegressor(
    activation = "identity",
    solver = "adam",
    learning_rate_init=0.001,    # Valor de la tasa de aprendizaje
    max_iter=1000,                     # Número máximo de iteraciones
)

grid_search = GridSearchCV(estimator = m, param_grid=param_grid, cv=5) 

## Entrenar con el mejor modelo
grid_search.fit(X_train, y_true_train)

#Predecir el comportamiento con el mejor modelo
y_pred = grid_search.predict(X_test)

## Calcular el error de la predicción
mse_val = mean_squared_error(y_true_test, y_pred)

# ---->>> Evaluación ---->>>
print(pytest.approx(mse_val, 0.0001) == 0.009535)
##

True
